In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "Norod78/english-sienfeld-distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint)


/home/mezzovilla/Projetos/magnitude/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
import os
project_path = Path().cwd().parent

model_path = project_path / checkpoint

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path)

('/home/mezzovilla/Projetos/magnitude/Norod78/english-sienfeld-distilgpt2/tokenizer_config.json',
 '/home/mezzovilla/Projetos/magnitude/Norod78/english-sienfeld-distilgpt2/special_tokens_map.json',
 '/home/mezzovilla/Projetos/magnitude/Norod78/english-sienfeld-distilgpt2/vocab.json',
 '/home/mezzovilla/Projetos/magnitude/Norod78/english-sienfeld-distilgpt2/merges.txt',
 '/home/mezzovilla/Projetos/magnitude/Norod78/english-sienfeld-distilgpt2/added_tokens.json',
 '/home/mezzovilla/Projetos/magnitude/Norod78/english-sienfeld-distilgpt2/tokenizer.json')

In [ ]:
print(f"Memory footprint: {model.get_memory_footprint() / 1e6:.2f} MB")

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

from pathlib import Path
project_path = Path().cwd().parent

model_name = 'distil_gpt2'
model_path = project_path 

tokenizer = AutoTokenizer.from_pretrained(model_path / f'tokenizers/{model_name}')
model = AutoModelForCausalLM.from_pretrained(model_path / f'models/{model_name}')

In [ ]:
inputs = tokenizer.encode("def print_hello_world():", return_tensors="pt")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

In [ ]:
type(tokenizer)

In [ ]:
phrases = [
    "The cat sleeps in the living room",
    "The cat runs in the living room",
    "The dog runs in the yard",
    "The cockroach flies in the house"
]

[tokenizer.tokenize(phrase) for phrase in phrases]

In [ ]:
import torch
import torch.nn.functional as F

def pi(model, tokenizer, x: str, y: str) -> float:
    if not y.startswith(x):
        return 0.0
    
    if x == y:
        return 1.0
    
    input_ids = tokenizer(x, return_tensors="pt").input_ids

    with torch.no_grad():
        out = model(input_ids)
        logits = out.logits[0, -1]

    probs = F.softmax(logits, dim=-1)

    y_id = tokenizer.encode(y, add_special_tokens=False)[0]
    p = probs[y_id]

    return p.item()

pi(model, tokenizer, x = 'Hello', y = 'World')
